# Sonnet + tf1 + tensorboard tutorial
https://qiita.com/n-suzuki/items/f6cb8bf5efff0f82d44d

In [1]:
import tensorflow as tf
import sonnet as snt

2022-04-03 03:04:03.808414: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0


In [2]:
# # モデル定義
# class MLP(snt.AbstractModule):
#     def __init__(self, output_size, nonlinearity=tf.nn.relu, name='mlp'):
#         super(MLP, self).__init__(name=name)
#         self._output_size = output_size
#         self._nonlinearity = nonlinearity

#     # 順伝播を記述
#     def _build(self, inputs, is_training):
#         conv1 = snt.Conv2D(output_channels=16, kernel_shape=3, stride=1, name='conv1')
#         h = self._nonlinearity(conv1(inputs))
#         h = snt.BatchNorm(name='bn1')(h, is_training=is_training)

#         conv2 = snt.Conv2D(output_channels=16, kernel_shape=3, stride=1, name='conv2')
#         h = self._nonlinearity(conv2(h))
#         h = snt.BatchNorm(name='bn2')(h, is_training=is_training)
#         h = snt.BatchFlatten(name='bf')(h)

#         l = snt.Linear(output_size=self._output_size, name='l')
#         outputs = tf.nn.softmax(l(h))

#         return outputs

# モデル定義
class MLP(snt.AbstractModule):
    def __init__(self, output_size, nonlinearity=tf.nn.relu, name='mlp'):
        super(MLP, self).__init__(name=name)
        self._output_size = output_size
        self._nonlinearity = nonlinearity

        # 学習パラメータ設定（_build()の内部で書くことも可）
        with self._enter_variable_scope():
            self._conv1 = snt.Conv2D(output_channels=16, kernel_shape=3, stride=1, name='conv1')
            self._bn1 = snt.BatchNorm(name='bn1')
            self._conv2 = snt.Conv2D(output_channels=16, kernel_shape=3, stride=1, name='conv2')
            self._bn2 = snt.BatchNorm(name='bn2')
            self._bf = snt.BatchFlatten(name='bf')
            self._l = snt.Linear(output_size=self._output_size, name='l')

    # 順伝播を記述（Chainerで言うところのforward(), __call__()）
    def _build(self, inputs, is_training):
        cnv1 = self._conv1
        h = self._nonlinearity(cnv1(inputs))
        h = self._bn1(h, is_training=is_training)

        cnv2 = self._conv2
        h = self._nonlinearity(cnv2(h))
        h = self._bn2(h, is_training=is_training)
        h = self._bf(h)

        l = self._l
        outputs = tf.nn.softmax(l(h))

        return outputs, h

In [3]:
# MNISTデータセット
from tensorflow.contrib.learn.python.learn.datasets.mnist import read_data_sets

# データセットのインスタンス（class MNISTに mnist.train, mnist.validation, mnist.testをそれぞれ渡す）
mnist = read_data_sets('./MNIST_data', one_hot=True)

# データセットモジュール定義
class MNIST(snt.AbstractModule):
    def __init__(self, mnist, batch_size, name='mnist'):
        super(MNIST, self).__init__(name=name)

        # データ数
        self._num_examples = mnist.num_examples
        # 画像(tf.constant)
        self._images = tf.constant(mnist.images, dtype=tf.float32)
        # ラベル(tf.constant)
        self._labels = tf.constant(mnist.labels, dtype=tf.float32)
        self._batch_size = batch_size

    def _build(self):
        # サンプラー(バッチ数)
        indices = tf.random_uniform([self._batch_size], 0, self._num_examples, tf.int64)
        x = tf.reshape(tf.gather(self._images, indices), (self._batch_size, 1, 28, 28))
        y_ = tf.gather(self._labels, indices)
        return x, y_

    def cost(self, logits, target):
        loss = -tf.reduce_sum(target * tf.log(logits))
        return loss

    def evaluation(self, logits, target):
        correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(target, 1))
        acc = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
        return acc

Extracting ./MNIST_data/train-images-idx3-ubyte.gz
Extracting ./MNIST_data/train-labels-idx1-ubyte.gz
Extracting ./MNIST_data/t10k-images-idx3-ubyte.gz
Extracting ./MNIST_data/t10k-labels-idx1-ubyte.gz


In [4]:
tf.app.flags.DEFINE_string('f', '', 'kernel')

In [5]:
# パラメータ設定
# TensorFlowと同じ

FLAGS = tf.flags.FLAGS

tf.flags.DEFINE_integer("num_training_iterations", 2000, "Number of iterations to train for.")
tf.flags.DEFINE_integer("report_interval", 50, "Iterations between reports (samples, valid loss).")
tf.flags.DEFINE_integer("batch_size", 64, "Batch size for training.")
tf.flags.DEFINE_integer("output_size", 10, "Size of MLP output layer.")

# データ供給用オブジェクトを作成
dataset_train = MNIST(mnist.train, batch_size=FLAGS.batch_size, name='mnist_train')
dataset_validation = MNIST(mnist.validation, batch_size=FLAGS.batch_size, name='mnist_validation')
dataset_test = MNIST(mnist.test, batch_size=FLAGS.batch_size, name='mnist_test')

# make MLP object
model = MLP(output_size=FLAGS.output_size, nonlinearity=tf.nn.relu)

# Build the training model and get the training loss.
# データ抽出
train_x, train_y_ = dataset_train()
# モデル順伝播
train_y, _ = model(train_x, True)
# loss計算
train_loss = dataset_train.cost(train_y, train_y_)
# lossをsummaryに保存（TensorBoard用）
tf.summary.scalar('loss', train_loss)

# Get the validation loss.
# データ抽出
validation_x, validation_y_ = dataset_validation()
# モデル順伝播
validation_y, _ = model(validation_x, False)
# loss計算
validation_loss = dataset_validation.evaluation(validation_y, validation_y_)
# validation accuracyをsummaryに保存（TensorBoard用）
tf.summary.scalar('validation_accuracy', validation_loss)

# Get the test loss.
# データ抽出
test_x, test_y_ = dataset_test()
# モデル順伝播
test_y, _ = model(test_x, False)
# loss計算
test_loss = dataset_test.evaluation(test_y, test_y_)
# test accuracyをsummaryに保存（TensorBoard用）
tf.summary.scalar('test_accuracy', test_loss)

# 中間層出力を取得(TensorBoard用)
_, features = model(tf.reshape(tf.constant(mnist.test.images, dtype=tf.float32), (-1, 1, 28, 28)), False)

# Set up optimizer.
train_step = tf.train.AdamOptimizer().minimize(train_loss)

In [6]:
from tensorboard.plugins import projector

In [7]:
# loggingレベルの設定
tf.logging.set_verbosity(tf.logging.INFO)

# training 
# モデル保存先ディレクトリ（=TensorBoardの--logdir）
log_dir = 'test_sonnet/'
with tf.Session() as sess:
    # パラメータ初期化
    sess.run(tf.global_variables_initializer())
    merged = tf.summary.merge_all()
    writer = tf.summary.FileWriter(log_dir, sess.graph)

    # メインループ
    for training_iteration in range(1, FLAGS.num_training_iterations + 1):
        # with validation
        if (training_iteration) % FLAGS.report_interval == 0:
            # train, validationのloss計算、optimize
            summary_, train_loss_v, validation_loss_v, _ = sess.run((merged, train_loss, validation_loss, train_step))
            # logging
            tf.logging.info("%d: Training loss %f. Validation accuracy %f.", training_iteration, train_loss_v, validation_loss_v)
            writer.add_summary(summary_, training_iteration)
        else:
            summary_, train_loss_v, _ = sess.run((merged, train_loss, train_step))
            writer.add_summary(summary_, training_iteration)
    test_loss_v = sess.run(test_loss)
    tf.logging.info("Test accuracy %f.", test_loss_v)

    # 中間層出力を取得してembedding_varとして保存
    feature = sess.run(features)
    embedding_var = tf.Variable(tf.stack([tf.squeeze(x) for x in feature], axis=0), trainable=False, name='feature')
    sess.run(tf.variables_initializer([embedding_var]))

    # モデルを保存
    saver = tf.train.Saver()
    saver.save(sess, log_dir + "model.ckpt")

    # TensorBoardの設定（PROJECTOR用）
    config = projector.ProjectorConfig()
    embedding = config.embeddings.add()
    embedding.tensor_name = embedding_var.name
    # メタデータのファイル名
    embedding.metadata_path = 'labels.tsv'
    # sprite画像のファイル名
    embedding.sprite.image_path = 'mnist_10k_sprite.png'
    # sprite画像の画像サイズ
    embedding.sprite.single_image_dim.extend([28, 28])

    projector.visualize_embeddings(writer, config)

# label to TSV 
import numpy as np
ls = np.argmax(mnist.test.labels, axis=1)

with open(log_dir + 'labels.tsv', 'w') as f:
    f.write('number\tlabel\n')
    for i, l in enumerate(ls):
        f.write('{}\t{}\n'.format(i, l))

2022-04-03 03:04:07.395414: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2022-04-03 03:04:07.442978: E tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:1068] could not open file to read NUMA node: /sys/bus/pci/devices/0000:15:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-04-03 03:04:07.443029: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1666] Found device 0 with properties: 
name: NVIDIA RTX A4000 major: 8 minor: 6 memoryClockRate(GHz): 1.56
pciBusID: 0000:15:00.0
2022-04-03 03:04:07.443548: E tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:1068] could not open file to read NUMA node: /sys/bus/pci/devices/0000:21:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-04-03 03:04:07.443596: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1666] Found device 1 with properties: 
name: Quadro P2000 major: 6 minor: 1 memoryClockRate(GHz): 1.4805
pciBusID: 0000

2022-04-03 03:04:08.949469: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1206] Device interconnect StreamExecutor with strength 1 edge matrix:
2022-04-03 03:04:08.949599: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1212]      0 1 
2022-04-03 03:04:08.949622: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1225] 0:   N N 
2022-04-03 03:04:08.949633: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1225] 1:   N N 
2022-04-03 03:04:08.951040: E tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:1068] could not open file to read NUMA node: /sys/bus/pci/devices/0000:15:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-04-03 03:04:08.951206: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1435] Could not identify NUMA node of platform GPU id 0, defaulting to 0.  Your kernel may not have been built with NUMA support.
2022-04-03 03:04:08.952089: E tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:1068] could not open file to read NUMA node: /sy

2022-04-03 03:04:12.787630: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.11
2022-04-03 03:04:13.967354: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudnn.so.8



INFO:tensorflow:50: Training loss 44.182106. Validation accuracy 0.875000.
INFO:tensorflow:100: Training loss 24.493500. Validation accuracy 0.890625.
INFO:tensorflow:150: Training loss 12.921508. Validation accuracy 0.843750.
INFO:tensorflow:200: Training loss 22.235863. Validation accuracy 0.937500.
INFO:tensorflow:250: Training loss 7.079167. Validation accuracy 0.984375.
INFO:tensorflow:300: Training loss 11.622228. Validation accuracy 0.921875.
INFO:tensorflow:350: Training loss 11.841982. Validation accuracy 0.968750.
INFO:tensorflow:400: Training loss 6.350767. Validation accuracy 0.968750.
INFO:tensorflow:450: Training loss 9.863495. Validation accuracy 0.968750.
INFO:tensorflow:500: Training loss 16.098486. Validation accuracy 0.953125.
INFO:tensorflow:550: Training loss 7.133687. Validation accuracy 0.968750.
INFO:tensorflow:600: Training loss 11.677952. Validation accuracy 0.953125.
INFO:tensorflow:650: Training loss 15.746884. Validation accuracy 0.968750.
INFO:tensorflow:

In [8]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

In [9]:
%tensorboard --logdir test_sonnet/

Launching TensorBoard...

AttributeError: module 'cgi' has no attribute 'escape'